#  ベクトルの内積・行列ベクトル積・行列積の区間演算

ベクトルの内積、行列ベクトル積、行列積の区間演算はとても重要。すなわち
$x, y\in\mathbb{IF}^n$, $A, B\in\mathbb{IF}^{n\times n}$ に対して、それぞれ$x^Ty \subset \alpha$, $Ax\subset z$, $AB\subset C$となる
$$
\alpha\in\mathbb{IF},\quad z\in\mathbb{IF}^n,\quad C\in\mathbb{IF}^{n\times n}
$$
を求める方法を考える。

## 素朴な方法

最も素朴な方法は各演算を区間演算に変更した方式である。

$$
    \alpha = \sum_{k = 1}^n x_ky_k
$$

$$
    z_i = \sum_{k = 1}^n A_{i,k}x_k\quad (1\le i\le n)
$$

$$
    C_{ij} = \sum_{k+1}^n A_{i,k}B_{k,j}\quad (1\le i,j\le n)
$$

特にJuliaの`IntervalArithmetic.jl`パッケージでは、この方法で区間演算が行なわれている。しかし、この方法だとプログラムの最適化が難しく、計算機の性能を十分に引き出した実装が難しく、計算時間がかかってしまう。

In [2]:
versioninfo()

Julia Version 1.11.5
Commit 760b2e5b739 (2025-04-14 06:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin24.0.0)
  CPU: 8 × Apple M2
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, apple-m2)
Threads: 4 default, 0 interactive, 2 GC (on 4 virtual cores)
Environment:
  JULIA_NUM_THREADS = 4


In [3]:
using Pkg
Pkg.status("IntervalArithmetic")

Status `~/.julia/environments/v1.11/Project.toml`
  [d1acc4aa] IntervalArithmetic v0.22.35


In [4]:
using LinearAlgebra
using IntervalArithmetic, BenchmarkTools

n = 1000
x = randn(n)
y = randn(n)

ix = interval(x)
iy = interval(y)
@btime alpha = dot($ix,$iy) # Level 1

# setrounding(Interval, :fast)

  44.000 μs (0 allocations: 0 bytes)


[12.0404, 12.0405]_com

In [5]:
n = 1000; A = randn(n,n); x = randn(n)
iA = interval(A); ix = interval(x); y = similar(ix)
@time z = iA * ix # Level 2
@time mul!(y, iA, ix) # Level 2

# ix = IntervalBox(ix)
# @time iz = iA * ix # Level 2

println(issubset_interval(y,z))
println(issubset_interval(z,y))

# @btime z = $A_int*$x_int;

  0.168998 seconds (630.81 k allocations: 31.796 MiB, 3.18% gc time, 64.04% compilation time)
  0.062098 seconds (72 allocations: 3.297 KiB, 3.74% compilation time)
true
true


In [6]:
n = 100
A = randn(n,n)
B = randn(n,n)
iA = interval(A)
iB = interval(B)
@time C = iA * iB  # Level 3
@time C = iA * iB  # Level 3
@time C = iA * iB; # Level 3

# A_int = SMatrix{n,n}(A_int);
# B_int = SMatrix{n,n}(B_int);
# @time C = A_int*B_int;  # Level 3
# @time C = A_int*B_int;
# @time C = A_int*B_int;

  0.151230 seconds (494.64 k allocations: 25.638 MiB, 59.93% compilation time)
  0.060364 seconds (3 allocations: 256.078 KiB)
  0.060519 seconds (3 allocations: 256.078 KiB)


100×100 Matrix{Interval{Float64}}:
  [-4.52848, -4.52847]_com    …  [-10.1517, -10.1515]_com
  [-5.90354, -5.90353]_com        [19.0921, 19.0922]_com
 [-10.3552, -10.3551]_com          [0.798259, 0.79826]_com
 [-10.111, -10.1109]_com          [15.4903, 15.4904]_com
  [-1.19862, -1.19861]_com        [23.243, 23.2431]_com
   [1.52761, 1.52762]_com     …  [-18.365, -18.3649]_com
 [-13.2879, -13.2878]_com          [6.86769, 6.8677]_com
 [-19.5613, -19.5611]_com         [-0.945912, -0.945911]_com
  [-0.922418, -0.922417]_com      [-1.75119, -1.75118]_com
  [-0.695091, -0.69509]_com      [-10.1601, -10.16]_com
    ⋮                         ⋱  
 [-16.1018, -16.1017]_com         [19.2508, 19.2509]_com
  [-0.366072, -0.366071]_com      [10.488, 10.4881]_com
   [1.95692, 1.95693]_com          [8.64718, 8.64719]_com
 [-11.1707, -11.1706]_com         [-7.44171, -7.4417]_com
   [2.48841, 2.48842]_com     …   [-6.99556, -6.99555]_com
   [5.98563, 5.98564]_com         [10.6551, 10.6552]_com
  [-5.684

このくらいの計算スピードならば、体感として、そこまで遅い印象はないが、そこそこ大きなサイズの行列積（Level 3 BLAS）は計算時間がかかってしまう。例えば、

In [7]:
n = 1000; A, B = randn(n,n), randn(n,n)
iA = interval(A)
iB = interval(B)
@time C = iA * iB;  # Level 3

 60.439004 seconds (3 allocations: 22.891 MiB)


**注意**　`BenchmarkTools`の`@btime`や`@benckmark`マクロを使うときは、
``
@btime A*B;
``
とせずに
``
@btime $A*$B;
``
と変数名に`$`をつける。これは計測時間に変数の呼び出しを含めるか指定するもので、前者が「変数の呼び出し時間を含めた計測」、後者は「行列積の計算時間の計測」を意味している。


さて、区間行列積(Level 3)を高速化したい。考えられる方法は

1. 並列化（[Loopvectrization](https://github.com/chriselrod/LoopVectorization.jl), [Tullio](https://github.com/mcabbott/Tullio.jl), `@simd`命令）

Tullioパッケージを使うと行列積を自動的に並列化して計算してくれる。小さいサイズの行列積はそこまで高速化されないが、ある程度行列サイズが大きくなると区間行列積の自動並列によって、計算時間が短くなることが観測される。ただし、`JULIA_NUM_THREADS`という環境変数で使用するスレッド数をあらかじめ定義しておく必要がある。例えばmacOSでzshというシェルを使っているなら、`~/.zshrc`というファイル内に

``export JULIA_NUM_THREADS=8``

と計算スレッド数を指定しておく必要がある。CPUのコア数分だけ指定したら良い？

In [8]:
# using LoopVectorization, BenchmarkTools, IntervalArithmetic, Tullio
using Tullio

tullio_gemm(A, B) = @tullio C[i,j] := A[i,k] * B[k,j]

n = 100
A, B = randn(n,n), randn(n,n)
iA = interval(A)
iB = interval(B)

@btime C = $iA * $iB;
@btime C1 = tullio_gemm($iA, $iB)

  60.186 ms (3 allocations: 256.08 KiB)
  15.703 ms (50 allocations: 258.55 KiB)


100×100 Matrix{Interval{Float64}}:
 [-11.6278, -11.6277]_com   …    [3.29631, 3.29632]_com
  [-1.162, -1.16199]_com         [2.92584, 2.92585]_com
  [13.2834, 13.2835]_com         [0.721624, 0.721625]_com
   [6.59471, 6.59472]_com        [7.34718, 7.34719]_com
   [7.96798, 7.96799]_com      [-18.1403, -18.1402]_com
  [10.8267, 10.8268]_com    …   [-8.28956, -8.28955]_com
  [-7.11751, -7.11749]_com      [-4.73135, -4.73134]_com
 [-10.4796, -10.4795]_com       [14.2798, 14.2799]_com
 [-11.2425, -11.2424]_com       [11.0072, 11.0073]_com
  [14.6567, 14.6568]_com         [4.09116, 4.09117]_com
    ⋮                       ⋱  
  [-8.62427, -8.62426]_com     [-15.6644, -15.6643]_com
   [8.26026, 8.26027]_com       [12.3283, 12.3284]_com
  [10.6095, 10.6096]_com        [-3.6928, -3.69279]_com
  [-2.0886, -2.08859]_com        [3.25317, 3.25318]_com
  [-3.50003, -3.50002]_com  …   [-4.1966, -4.19659]_com
  [-3.45124, -3.45123]_com       [0.76803, 0.768031]_com
 [-11.7032, -11.7031]_com        [9

In [9]:
# Threads.nthreads()
n = 1000;
iA, iB = interval(randn(n,n)), interval(randn(n,n))
# @btime C = A*B; # ≈ 40 sec
# setrounding(Interval, :tight)
@btime C1 = tullio_gemm(iA,iB); # ≈ 6 sec

  16.167 s (50 allocations: 22.89 MiB)


次、行列、ベクトルの配列に

2. [StaticArrays](https://github.com/JuliaArrays/StaticArrays.jl_)を使うと[速いらしい](https://qiita.com/cometscome_phys/items/669273a49ab2417d3af8)

`StaticArrays`は小さいサイズでは多少計算時間が変わるが、要素数が多くなるほどコンパイルに時間がかかり、全体の経過時間は長くなってしまった。要素数が100以下なら`StaticArrays`が使えると[Webサイト](https://github.com/JuliaArrays/StaticArrays.jl_)に書いてありました。

残るは

3. BLAS-Level 3を使う区間演算を実装する、だけど丸め方向が変えられないので、丸めの向きを変えない区間演算を実装する必要あり
    - 内積は文献3.の定理8 (今回は実装しない)
    - 行列ベクトル積は？？ (今回は実装しない)
    - 行列積は文献3.の4節、アルゴリズム5

 
 ## BLASを使う
 
 まずJuliaの中で、[BLAS](https://ja.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms)を使うには``LinearAlgebra``パッケージが必要。

In [10]:
using LinearAlgebra, BenchmarkTools
# versioninfo()
# BLAS.vendor()
n = 5000
A, B, C = randn(n,n), randn(n,n), zeros(n,n)

# @btime C = $A * $B;
@btime mul!(C, A, B);

  1.628 s (0 allocations: 0 bytes)


In [11]:
# using IntervalArithmetic, MKL
# n = 65
# A, B = randn(n,n), randn(n,n)
# setrounding(Float64, RoundUp)
# C_up = A * B
# setrounding(Float64, RoundDown)
# C_down = A * B
# setrounding(Float64, RoundNearest)
# C_down .< C_up

このようにBLASを使うと、物凄いスピードで数値計算ができる。ただし、行列の内部の型がFloat64に限られるのと、丸め方向を変更した計算ができないため、区間演算の結果が粗くなる（区間幅が増加する）。以下、**丸め方向を変更しない区間行列積**を実装する。

**定義**　$\mathbf{u}=2^{-53}$を倍精度の**単位相対丸め**とする。$\mathbf{S}_{\min}=2^{-1074}$を倍精度浮動小数点数の正の最小数とする。$\mathbf{F}_{\min}=2^{-1022}$を倍精度浮動小数点数の正規化数の正の最小数とする。 

**注意**　単位相対丸めは$1$と$1$よりも小さい最大の浮動小数点数との差を表す。つまり

$$
1-\mathbf{u}<a<1
$$

となる$a\in\mathbb{F}$は存在しない。

**注意**　単位相対丸めとは別に**計算機イプシロン**（unit roundoff, machine epsilon）という単位もあり、こちらは$1$と$1$よりも大きい最小の浮動小数点数との差を表し、64bit浮動小数点数では$2^{-52}$でこちらを$\mathbf{u}$と書く流儀もある（こっちが主流？）ので、注意が必要。Juliaでは`eps(Float64)`とすると$2^{-52}$の数値が得られる。

In [12]:
u = 2.0^-53;
s_min = 2.0^-1074;
f_min = 2.0^-1022;
println(bitstring(1.0))
println(bitstring(1.0-u))
println(bitstring(s_min))
println(bitstring(f_min))
println(eps(Float64))
println(2*u)

0011111111110000000000000000000000000000000000000000000000000000
0011111111101111111111111111111111111111111111111111111111111111
0000000000000000000000000000000000000000000000000000000000000001
0000000000010000000000000000000000000000000000000000000000000000
2.220446049250313e-16
2.220446049250313e-16


**定義**　実数$a\in\mathbb{R}$に対する**U**nit in the **F**irst **P**lace (ufp)は次のように定める。

$$
 \text{ufp}(a):= 2^{\lfloor\log_2|a|\rfloor} (a\neq 0),\quad \text{ufp}(0)=0.
$$

関数 $\text{ufp}$ は実数$a$を2進数表記した先頭ビットを返す。4回の浮動小数点演算で求めることができる。関数 $\text{ufp}$ を使うと $\text{ufp}(a)\le a < 2\text{ufp}$ が成立する。

In [13]:
function ufp(P)
    u = 2.0^(-53);
    ϕ = 2.0^52 + 1;
    q = ϕ * P;
    T = (1 - u)*q;
    return abs(q - T);
end

ufp (generic function with 1 method)

In [14]:
p = 1.0 - u;
println(bitstring(p))
println(bitstring(ufp(p)))
println(bitstring(2*ufp(p)))

0011111111101111111111111111111111111111111111111111111111111111
0011111111100000000000000000000000000000000000000000000000000000
0011111111110000000000000000000000000000000000000000000000000000


**定義**　$c\in\mathbb{R}$として、

- $\text{pred}$: $c$ より小さい最大の浮動小数点数を返す関数, $\text{pred}(c):=\max\{f\in \mathbb{F}:f<c\}$
- $\text{succ}$: $c$ より大きい最小の浮動小数点数を返す関数, $\text{succ}(c):=\min\{f\in \mathbb{F}:c<f\}$


これらを使うと $a$, $b\in\mathbb{F}$, $\circ\in\{+,-,\times,\div\}$ で

$$
    \text{pred}(\text{fl}(a\circ b))<a\circ b<\text{succ}(\text{fl}(a\circ b))
$$

が成り立つ。ここで $\text{fl}:\mathbb{R}\to\mathbb{F}$ は入力の実数を最も近い浮動小数点数に写す写像で、$\text{fl}$ 内の演算は全て浮動小数点演算で計算されているとする。

また、pred関数, succ関数はベクトル・行列においても各要素に対するsucc, predを考える事で拡張することが出来る。以下では、文献1のRumpによる方法を実装する。これは複雑だが、なるべく非正規化数の演算を避けるように設計されている。

In [15]:
function succ(c)
    s_min = 2.0^-1074;
    u = 2.0^-53;
    ϕ = u * (1.0 + 2.0 * u);
    if abs(c) >= (1. / 2.) * u^(-2) * s_min # 2^(-969)(Float64)
        e = ϕ * abs(c);
        succ = c + e;
    elseif abs(c) < (1. / 2.) * u^(-1) * s_min # 2^(-1022)(Float64)
        succ = c + s_min;
    else
        C = u^(-1) * c;
        e = ϕ * abs(C);
        succ = (C + e) * u;
    end
    return succ
end

function pred(c)
    s_min = 2.0^-1074;
    u = 2.0^-53;
    ϕ = u * (1.0 + 2.0 * u);
    if abs(c) >= (1. / 2.) * u^(-2) * s_min # 2^(-969)(Float64)
        e = ϕ * abs(c);
        pred = c - e;
    elseif abs(c) < (1. / 2.) * u^(-1) * s_min # 2^(-1022)(Float64)
        pred = c - s_min;
    else
        C = u^(-1) * c;
        e = ϕ * abs(C);
        pred = (C - e) * u;
    end
    return pred
end

pred (generic function with 1 method)

In [16]:
a = 0.1;
println(bitstring(succ(a)))
println(bitstring(a))
println(bitstring(pred(a)))

0011111110111001100110011001100110011001100110011001100110011011
0011111110111001100110011001100110011001100110011001100110011010
0011111110111001100110011001100110011001100110011001100110011001


ベクトルの総和と内積のufpを用いた事前誤差解析を応用して、行列 $A\in\mathbb{F}^{m\times n}$, $B\in\mathbb{F}^{n\times p}$ の積 $AB$ を包含する中心半径型区間行列 $\langle C,R\rangle\in\mathbb{IF}^{m\times p}$ を計算する。

$$
    C=\text{fl}(AB),\quad R=\text{fl}((n+2)\mathbf{u}\cdot \text{ufp}(|A||B|)) + \mathbf{F}_{\min}\cdot e_{m}e_{p}^T).
$$

ただし $e_n=(1,\dots,1)^T\in\mathbb{F}^{n}$ で $n$ は $2(n+\mathbf{u})<1$を満たすとする。このとき必要な行列積の計算は、$\text{fl}(AB)$, $\text{fl}(|A||B|)$ の2回である。これらをBLASを用いて計算することにより、高速な数値計算が実行可能である。そして、浮動小数点演算を行った結果 $C$ とその誤差半径を $R$ とし、行列積の真値を包含する。

In [17]:
function mm_ufp(A_mid, B_mid)
    u = 2.0^(-53);
    realmin = 2.0^(-1022);
    n = size(A_mid,2);
    
    if(2*(n+2)*u>=1)
        error("mm_ufp is failed!(2(n+2)u>=1)")
    end
    C_mid = A_mid * B_mid;
    C_rad = (n+2) * u * ufp.(abs.(A_mid)*abs.(B_mid)) .+ realmin;
        
    return C_mid, C_rad;
end

mm_ufp (generic function with 1 method)

In [ ]:
using IntervalArithmetic, LinearAlgebra
# using IntervalArithmetic.Symbols
n = 1000
A = randn(n,n)
B = randn(n,n)
iA = interval(A)
iB = interval(B)

A_mid = mid.(iA)
B_mid = mid.(iB)

@btime C_mid, C_rad = mm_ufp(A_mid, B_mid)
@time C_mid, C_rad = mm_ufp(A_mid, B_mid);
C_int = interval(C_mid, C_rad; format=:midpoint);

  26.459 ms (22 allocations: 53.59 MiB)
  0.034732 seconds (24 allocations: 53.594 MiB, 3.97% gc time)


**注意** `@tullio`マクロで並列化した区間演算（8コア！）で約6秒かかる区間行列積が、BLASを使うと、約0.02秒と300倍程度高速化される。これがBLASの力だー！しかし、BLASの計算はFloat64のみサポートされるため、64ビットの浮動小数点数を用いた区間演算のとき以外は、このままでは高速化されない点を注意したい。例えば端点に`BigFloat`のような多倍長数値を用いたいときなどは、残念ながらBLASを用いたの高速な計算が出来ないのが現状である。また、ufpを用いる区間演算は過大評価になるため、区間幅の増大が避けられない事（概ね10倍大きな区間幅になる）にも注意が必要である。

In [25]:
C1 = tullio_gemm(iA, iB);
println(maximum(radius.(C1[:])))
println(maximum(C_rad[:]))

9.492850949754938e-12
5.695710569852963e-11


次に区間行列 $A=\langle A_m,A_r\rangle\in\mathbb{IF}^{m\times n}$, $B=\langle B_m,B_r\rangle\in\mathbb{IF}^{n\times p}$ の積 $AB$ を包含する区間行列 $\langle C,T\rangle\in\mathbb{IF}^{m\times p}$ を計算することを考える。はじめに区間行列積 $AB$ は

$$
    AB\subseteq \langle A_mB_m,\tilde{T}\rangle,\quad \tilde{T}:=|A_m|B_r+A_r(|B_m|+B_r)
$$

と包含できる。丸め方向の変更ができる場合は、この中心と半径を丸めの向きを変えることで計算し厳密に包含することができる。しかし今回は丸め方向の変更ができないため、事前誤差評価によって中心と半径を厳密に包含する必要がある。いま $n$ は $2(n+\mathbf{u})<1$ を満たすとし、区間行列の中心同士の積 $A_mB_m$ は

$$
    C=\text{fl}(A_mB_m),\quad R=\text{fl}((n+2)\mathbf{u}\cdot \text{ufp}(|A_m||B_m|)) + \mathbf{F}_{\min}\cdot e_{m}e_{p}^T).
$$

として $\langle C,R\rangle(=\texttt{mm\_ufp}(A_m,B_m))$ で包含できる。次に $|A_m|B_r$ の上限を考える。

\begin{align*}
&T_1:=\text{fl}(|A_m|B_r)\\
&||A_m|B_r-\text{fl}(|A_m|B_r)|\le \text{fl}((n+2)\mathbf{u}\cdot\text{ufp}(T_1) + \mathbf{F}_{\min}\cdot e_{m}e_{p}^T)=:T_2\\
&|A_m|B_r\le T_1+T_2
\end{align*}

さらに $|B_m|+B_r$ は

$$
    |B_m|+B_r\le\text{succ}(\text{fl}(|B_m|+B_r))=:T_3
$$

で抑えられる。さらに $A_rT_3$ の上限は

\begin{align*}
&T_4:=\text{fl}(A_rT_3)\\
&|A_rT_3-\text{fl}(A_rT_3)|\le \text{fl}((n+2)\mathbf{u}\cdot\text{ufp}(T_4) + \mathbf{F}_{\min}\cdot e_{m}e_{p}^T)=:T_5\\
&A_rT_3\le T_4+T_5.
\end{align*}

最後に誤差半径 $T$ は $T_1+T_2+T_3+T_4+T_5+R$の上限で得られ、

\begin{align*}
&T_1+T_2+T_3+T_4+T_5+R\\
&\le \text{fl}(T_1+T_2+T_3+T_4+T_5+R) + \text{fl}(4\mathbf{u}(T_1+T_2+T_3+T_4+T_5+R))\\
&\le \text{succ}(\text{fl}(T_1+T_2+T_3+T_4+T_5+R) + \text{fl}(4\mathbf{u}(T_1+T_2+T_3+T_4+T_5+R)))=:T.
\end{align*}

したがって、$AB\subseteq \langle C,T\rangle$ となる $C\in\mathbb{F}^{m\times p}$ と $T\in\mathbb{F}^{m\times p}$ が得られる。

In [20]:
function imm_ufp(A_mid, A_rad, B_mid, B_rad)
    u = 2.0^(-53);
    realmin = 2.0^(-1022);
    n = size(A_mid,2);
    
    if(2*(n+2)*u>=1)
        error("mm_ufp is failed!(2(n+2)u>=1)")
    end
#     C, R = mm_ufp(A_mid,B_mid);
    C_mid = A_mid * B_mid;
    R = (n+2) * u * ufp.(abs.(A_mid)*abs.(B_mid)) .+ realmin;
    
#     T_1, T_2 = mm_ufp(abs.(A_mid), B_rad);
    T1 = abs.(A_mid) * B_rad;
    T2 = (n+2)*u*ufp.(T1) .+ realmin;
    
#     T_3 = succ.(abs.(B_mid)+B_rad);
    T3 = succ.(abs.(B_mid)+B_rad)
    
#     T_4, T_5 = mm_ufp(A_r, T_3);
    T4 = A_rad * T3;
    T5 = (n+2)*u*ufp.(T4) .+ realmin;
    
    rad_sum = R + T1 + T2 + T4 + T5;

    C_rad = succ.(rad_sum + 4*u*ufp.(rad_sum));
    
    return C_mid, C_rad;
end

imm_ufp (generic function with 1 method)

In [26]:
using IntervalArithmetic, LinearAlgebra

# n = 1000;
# A = randn(n,n);
# B = randn(n,n);
# A_int = map(Interval, A);
# B_int = map(Interval, B);

A_mid = mid.(iA);
A_rad = radius.(iA);
B_mid = mid.(iB);
B_rad = radius.(iB);

@btime C1_mid, C1_rad = imm_ufp(A_mid, A_rad, B_mid, B_rad);
C1_mid, C1_rad = imm_ufp(A_mid,A_rad,B_mid,B_rad);
C1_int = interval(C1_mid, C1_rad; format=:midpoint);


  64.713 ms (73 allocations: 183.75 MiB)


In [22]:
# CC = A_mid*B_mid .∈ C1;
# sum(CC[:])
# println(size(C_int))
# println(size(C1))
# println(maximum(radius.(C1[:])))
println(maximum(C_rad[:]))
println(maximum(C1_rad[:]))
# println(size(C_int))

5.695710569852963e-11
5.695710569852965e-11


In [23]:
include("IntervalFunctions.jl")
n = 1000
A, B = randn(n,n), randn(n,n)
iA, iB = interval(A), interval(B)
@time iC = int_mul(iA, iB);

  0.443668 seconds (595.64 k allocations: 275.514 MiB, 3.45% gc time, 63.26% compilation time)


**注意**　`mm_ufp`と`imm_ufp`は入力の行列 $A$, $B$ に応じて使い分けることを計算速度、誤差半径の両面からおすすめする。

### 謝辞

本資料も筆者が学生の頃に精度保証付き数値計算を教えて下さった[柏木雅英](http://www.kashi.info.waseda.ac.jp/~kashi/)先生の「数値解析特論」の講義資料を参考にしています。また、丸め方向を変更しないBLASを使った区間行列積について[森倉悠介](http://y-m.jp/)先生から助言をいただき、`imm_ufp`のMATLABコードをご提供していただきました。ここに感謝申し上げます。
さらに、以下のような文献・Web ページ等を参考にこの文章は書いています。


### 参考文献
1. S. M. Rump, P. Zimmermann, S. Boldo and G. Melquiond: “Computing predecessor and successor in rounding to nearest”, BIT Vol. 49, No. 2, pp.419–431, 2009.
(http://www.ti3.tu-harburg.de/paper/rump/RuZiBoMe08.pdf)
(succ, predのRumpの方法が紹介されている)
1. 柏木雅英, IEEE754 浮動小数点数の隣の数を計算する方法のまとめ, [http://www.kashi.info.waseda.ac.jp/~kashi/lec2020/nac/succ.pdf](http://www.kashi.info.waseda.ac.jp/~kashi/lec2020/nac/succ.pdf)
(succ, predの様々な実装方法が紹介されている)
1. 森倉悠介, 事前誤差評価を用いた線形計算の精度保証 –誤差解析から大規模計算まで–, [http://www.sr3.t.u-tokyo.ac.jp/jsiam/slides/2015/morikura_20151224_yokou_ver1.3.pdf](http://www.sr3.t.u-tokyo.ac.jp/jsiam/slides/2015/morikura_20151224_yokou_ver1.3.pdf)
(丸め方向を変更しない数値線形代数の区間演算について端的にまとまっている)
1. 大石進一編著, 精度保証付き数値計算の基礎, コロナ社, 2018.<br>
(精度保証付き数値計算の教科書。2章に浮動小数点数の事前誤差解析の話が、3.2章に区間行列積の高速実装についての話がそれぞれ載っている。ここでは紹介しなかったが、この先にエラーフリー変換という沼が待っている。ぜひご体感あれ！)


<div align="right"><a href="http://www.risk.tsukuba.ac.jp/~takitoshi/">高安亮紀</a>，2020年9月25日（最終更新：2025年5月26日）</div>